## Imports

In [1]:
import numpy as np
import pandas as pd

import catboost as cb

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression

import optuna

In [2]:
cb.__version__

'1.1.1'

In [3]:
train = pd.read_csv('data/train.csv')
train.head()

,map_id,team1_id,team2_id,map_name,who_win
0,289,6665,7718,Ancient,0
1,715,4411,10577,Inferno,0
2,157,11251,9455,Nuke,1
3,524,4608,7532,Mirage,0
4,404,8637,6667,Overpass,1


train.csv - история сыгранных карт между двумя командами. 
* `map_id`- уникальный идентификатор сыгранной карты
* `team1_id`- уникальный идентификатор команды
* `team2_id`- уникальный идентификатор команды
* `map_name`- название карты
* `who_win` - переменная, которую хотим прогнозировать. Значение 0 означает, что победила команда `team1_id`, значение 1 - победила команда `team2_id`.


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   map_id    713 non-null    int64 
 1   team1_id  713 non-null    int64 
 2   team2_id  713 non-null    int64 
 3   map_name  713 non-null    object
 4   who_win   713 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 28.0+ KB


In [5]:
train['map_id'].count()

713

In [6]:
train['team1_id'].nunique()

52

In [7]:
train['team2_id'].nunique()

59

In [8]:
train['map_name'].nunique()

7

In [9]:
train['map_name'].value_counts()

Mirage      118
Inferno     117
Nuke        109
Overpass    100
Ancient      99
Dust2        92
Vertigo      78
Name: map_name, dtype: int64

In [10]:
train['who_win'].value_counts()

0    386
1    327
Name: who_win, dtype: int64

In [11]:
players_feats = pd.read_csv('data/players_feats.csv')
players_feats.head()

,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,...,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id,map_name,map_id
0,4954,90,42.2,112,0.80,76.3,5.9,6,156,0.58,...,5,25,12,2.08,1.28,84.0,25.0,6665,Ancient,635
1,5794,45,60.0,57,0.79,82.3,10.9,3,68,0.66,...,96,54,34,1.59,1.17,70.4,16.7,7532,Ancient,635
2,4954,156,51.9,167,0.93,63.5,3.4,10,265,0.59,...,22,26,19,1.37,1.10,88.5,20.5,6665,Dust2,583
3,5794,449,53.5,427,1.05,86.7,13.1,23,618,0.73,...,104,62,49,1.27,1.10,79.0,17.4,7532,Dust2,583
4,7998,173,32.9,130,1.33,82.4,2.9,9,225,0.77,...,19,27,25,1.08,1.08,81.5,16.2,4608,Dust2,439


`players_feats.csv` - признаки игроков, взятых из статистики сайта HLTV за полугодовой период для каждого `map_id` из файлов `train.csv` и `test.csv`. То есть, одна строка является агрегацией за полгода признаков игроков команды `team_id` для карты `map_id` (если дата матча 2022-01-10, то признаки будут сумма/среднее за период с 2021-06-10 до 2022-01-09). 

Названия колонок состоят из номера игрока и названия метрики. 
Например, признак `p1_total_kills` означает суммарное количество убийств, совершенных игроком `p1` за полгода до даты карты `map_id`. 

Вы можете использовать эти признаки, чтобы как то оценить силу команд. 

Файлы `train.csv` и `players_feats.csv` можно смерджить по `train.map_id == players_feats.map_id and train.team{1|2}_id == players_feats.team_id`.


In [12]:
players_feats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1486 entries, 0 to 1485
Columns: 128 entries, p1_id to map_id
dtypes: float64(80), int64(47), object(1)
memory usage: 1.5+ MB


In [13]:
players_feats.columns[:25]

Index(['p1_id', 'p1_total_kills', 'p1_headshots', 'p1_total_deaths',
       'p1_kd_ratio', 'p1_damage_per_round', 'p1_grenade_damage_per_round',
       'p1_maps_played', 'p1_rounds_played', 'p1_kills_per_round',
       'p1_assists_per_round', 'p1_deaths_per_round',
       'p1_saved_by_teammate_per_round', 'p1_saved_teammates_per_round',
       'p1_rating', 'p1_kill_death', 'p1_kill_round', 'p1_rounds_with_kills',
       'p1_kill_death_difference', 'p1_total_opening_kills',
       'p1_total_opening_deaths', 'p1_opening_kill_ratio',
       'p1_opening_kill_rating', 'p1_team_win_percent_after_first_kill',
       'p1_first_kill_in_won_rounds'],
      dtype='object')

In [14]:
players_feats['map_id']

0       635
1       635
2       583
3       583
4       439
       ... 
1481    284
1482     27
1483     27
1484    237
1485    237
Name: map_id, Length: 1486, dtype: int64

## CV func

In [62]:
def cv_func(X,y,cat_features,n_trials=100,
            plot_best=True,
           return_best=False):
    
    
    
    def objective(trial):

        param = {
            "depth": trial.suggest_int("depth", 2, 6),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            # 'loss_function': trial.suggest_categorical("loss_function", ["RMSE", "MAE"]),
            'loss_function': 'Logloss',
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000, log=True),
            'custom_loss': 'AUC'
        }

        skf = StratifiedKFold(n_splits=8, random_state=1,shuffle=True)

        scores = []

        for train_index,test_index in skf.split(X,y):

            X_train,y_train = X.loc[train_index],y.loc[train_index]
            X_test,y_test = X.loc[test_index],y.loc[test_index]

            model = cb.CatBoostClassifier(**param, verbose=0)

            dataset = cb.Pool(X_train,y_train,cat_features=cat_features)
            eval_dataset = cb.Pool(X_test,y_test,cat_features=cat_features)

            model.fit(dataset, eval_set=eval_dataset)

            roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
            scores.append(roc_auc)

        print(scores)
        return sum(scores)/8
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, timeout=600, n_jobs=-1)
    
    print('Best params:',study.best_params)
    print('Best roc_auc:',study.best_value)
    if plot_best:
        params = study.best_params

        params['loss_function'] = 'Logloss'
        params['custom_loss'] = 'AUC'
        params["verbose"] = False

        dataset = cb.Pool(X,y,cat_features=cat_features)

        scores = cb.cv(dataset,
                        params,
                        fold_count=8, 
                        plot="True")
        
    if return_best:
        return study.best_value,study.best_params

In [16]:
# def cv_func(X,y,cat_features,n_trials=100):
    
    
    
#     def objective(trial):

#         param = {
#             "depth": trial.suggest_int("depth", 2, 6),
#             'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
#             # 'loss_function': trial.suggest_categorical("loss_function", ["RMSE", "MAE"]),
#             'loss_function': 'Logloss',
#             'n_estimators': trial.suggest_int('n_estimators', 100, 1000, log=True),
#             'custom_loss': 'AUC'
#         }



#         dataset = cb.Pool(X,y,cat_features=cat_features)
            
#         scores = cb.cv(dataset,
#                     param,
#                     fold_count=8, 
#                     plot="False",
#                     verbose=-1)

#         return scores.iloc[-1,-2]
    
#     study = optuna.create_study(direction='maximize')
#     study.optimize(objective, n_trials=n_trials, timeout=600, n_jobs=-1)
    
#     print('Best params:',study.best_params)
#     print('Best roc_auc:',study.best_value)
    
#     params = study.best_params
    
#     params['loss_function'] = 'Logloss'
#     params['custom_loss'] = 'AUC'
#     params["verbose"] = False
    
#     dataset = cb.Pool(X,y,cat_features=cat_features)
           
#     scores = cb.cv(dataset,
#                     params,
#                     fold_count=8, 
#                     plot="True")

In [17]:
# def objective(trial):

#     param = {
#         "depth": trial.suggest_int("depth", 2, 6),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
#         # 'loss_function': trial.suggest_categorical("loss_function", ["RMSE", "MAE"]),
#         'loss_function': 'Logloss',
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000, log=True),
#         'custom_loss': 'AUC'
#     }

    
#     skf = StratifiedKFold(n_splits=8, random_state=1,shuffle=True)
    
#     scores = []
    
#     for train_index,test_index in skf.split(X,y):
        
#         X_train,y_train = X.loc[train_index],y.loc[train_index]
#         X_test,y_test = X.loc[test_index],y.loc[test_index]
        
#         model = cb.CatBoostClassifier(**param, verbose=0)
        
#         dataset = cb.Pool(X_train,y_train,cat_features=cat_features)
#         eval_dataset = cb.Pool(X_test,y_test,cat_features=cat_features)
        
#         model.fit(dataset, eval_set=eval_dataset)

#         roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
#         scores.append(roc_auc)
    
    

#     return sum(scores)/8

# %%time
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100, timeout=600, n_jobs=-1)

# study.best_params

# study.best_value

# params = {"iterations": 538,
#           "depth": 2,
#           "loss_function": "Logloss",
#           "verbose": False,
#           'custom_loss': 'AUC',
#          'learning_rate': 0.045525831554551505}

# scores = cb.cv(dataset,
#             params,
#             fold_count=5, 
#             plot="True")

## 1 First df merge as is

In [18]:
df = train.merge(players_feats.add_suffix('_team1'),left_on=['map_id','team1_id'],right_on=['map_id_team1','team_id_team1']).merge(players_feats.add_suffix('_team2'),left_on=['map_id','team2_id'],right_on=['map_id_team2','team_id_team2'])


In [19]:
cols_to_drop = ['who_win','map_id',
                'map_id_team1','map_id_team2',
                'team_id_team1','team_id_team2',
                'map_name_team1','map_name_team2',
               'p1_id_team1','p1_id_team2',
               'p2_id_team1','p2_id_team2',
               'p3_id_team1','p3_id_team2',
               'p4_id_team1','p4_id_team2',
               'p5_id_team1','p5_id_team2']

In [20]:
X = df.drop(columns=cols_to_drop)
y = df['who_win']

In [21]:
cat_features = ['team1_id','team2_id','map_name']

In [22]:
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

### Catboost cv results with optuna

In [23]:
# cv_func(X,y,cat_features,4)

## 2 (stats sum)

In [24]:
players_feats

,p1_id,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,...,p5_kill_death_difference,p5_total_opening_kills,p5_total_opening_deaths,p5_opening_kill_ratio,p5_opening_kill_rating,p5_team_win_percent_after_first_kill,p5_first_kill_in_won_rounds,team_id,map_name,map_id
0,4954,90,42.2,112,0.80,76.3,5.9,6,156,0.58,...,5,25,12,2.08,1.28,84.0,25.0,6665,Ancient,635
1,5794,45,60.0,57,0.79,82.3,10.9,3,68,0.66,...,96,54,34,1.59,1.17,70.4,16.7,7532,Ancient,635
2,4954,156,51.9,167,0.93,63.5,3.4,10,265,0.59,...,22,26,19,1.37,1.10,88.5,20.5,6665,Dust2,583
3,5794,449,53.5,427,1.05,86.7,13.1,23,618,0.73,...,104,62,49,1.27,1.10,79.0,17.4,7532,Dust2,583
4,7998,173,32.9,130,1.33,82.4,2.9,9,225,0.77,...,19,27,25,1.08,1.08,81.5,16.2,4608,Dust2,439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,9031,69,49.3,70,0.99,69.8,5.7,4,114,0.61,...,-1,16,18,0.89,1.11,75.0,21.4,10503,Ancient,284
1482,7716,104,55.8,113,0.92,71.6,5.9,6,172,0.60,...,4,33,34,0.97,1.30,75.8,28.4,7020,Vertigo,27
1483,12521,141,37.6,159,0.89,79.7,7.4,8,210,0.67,...,5,9,15,0.60,0.77,55.6,4.3,8297,Vertigo,27
1484,7716,155,58.7,154,1.01,71.1,7.9,10,246,0.63,...,35,30,17,1.76,1.39,83.3,22.9,7020,Ancient,237


In [25]:
128/5

25.6

In [26]:
players_feats.columns[:26]

Index(['p1_id', 'p1_total_kills', 'p1_headshots', 'p1_total_deaths',
       'p1_kd_ratio', 'p1_damage_per_round', 'p1_grenade_damage_per_round',
       'p1_maps_played', 'p1_rounds_played', 'p1_kills_per_round',
       'p1_assists_per_round', 'p1_deaths_per_round',
       'p1_saved_by_teammate_per_round', 'p1_saved_teammates_per_round',
       'p1_rating', 'p1_kill_death', 'p1_kill_round', 'p1_rounds_with_kills',
       'p1_kill_death_difference', 'p1_total_opening_kills',
       'p1_total_opening_deaths', 'p1_opening_kill_ratio',
       'p1_opening_kill_rating', 'p1_team_win_percent_after_first_kill',
       'p1_first_kill_in_won_rounds', 'p2_id'],
      dtype='object')

In [27]:
players_feats.columns[-30:]

Index(['p4_team_win_percent_after_first_kill', 'p4_first_kill_in_won_rounds',
       'p5_id', 'p5_total_kills', 'p5_headshots', 'p5_total_deaths',
       'p5_kd_ratio', 'p5_damage_per_round', 'p5_grenade_damage_per_round',
       'p5_maps_played', 'p5_rounds_played', 'p5_kills_per_round',
       'p5_assists_per_round', 'p5_deaths_per_round',
       'p5_saved_by_teammate_per_round', 'p5_saved_teammates_per_round',
       'p5_rating', 'p5_kill_death', 'p5_kill_round', 'p5_rounds_with_kills',
       'p5_kill_death_difference', 'p5_total_opening_kills',
       'p5_total_opening_deaths', 'p5_opening_kill_ratio',
       'p5_opening_kill_rating', 'p5_team_win_percent_after_first_kill',
       'p5_first_kill_in_won_rounds', 'team_id', 'map_name', 'map_id'],
      dtype='object')

In [28]:

p1 = pd.concat([players_feats.filter(regex='^p1_',axis=1),players_feats[['map_id','map_name','team_id']]],axis=1)
p2 = pd.concat([players_feats.filter(regex='^p2_',axis=1),players_feats[['map_id','map_name','team_id']]],axis=1)
p3 = pd.concat([players_feats.filter(regex='^p3_',axis=1),players_feats[['map_id','map_name','team_id']]],axis=1)
p4 = pd.concat([players_feats.filter(regex='^p4_',axis=1),players_feats[['map_id','map_name','team_id']]],axis=1)
p5 = pd.concat([players_feats.filter(regex='^p5_',axis=1),players_feats[['map_id','map_name','team_id']]],axis=1)

col_names = p1.columns.tolist()
temp = [x[3:] for x in p1.columns[:-3]]
col_names[:-3] = temp

for p in [p1,p2,p3,p4,p5]:
    p.columns = col_names

In [29]:
# col_names[1:-3]

In [30]:
# p1['kill_death']

In [31]:
# p1['kill_round']

In [32]:
# p1['kill_death_difference']

In [33]:
# p1['opening_kill_rating']

In [34]:
stats_sum = p1[col_names[1:-3]] + p2[col_names[1:-3]] + p3[col_names[1:-3]] + p4[col_names[1:-3]] + p5[col_names[1:-3]]

In [35]:
stats_sum = pd.concat([stats_sum,players_feats[['map_id','map_name','team_id']]],axis=1)

In [36]:
stats_sum

,total_kills,headshots,total_deaths,kd_ratio,damage_per_round,grenade_damage_per_round,maps_played,rounds_played,kills_per_round,assists_per_round,...,kill_death_difference,total_opening_kills,total_opening_deaths,opening_kill_ratio,opening_kill_rating,team_win_percent_after_first_kill,first_kill_in_won_rounds,map_id,map_name,team_id
0,492,224.8,557,4.38,357.7,28.2,32,814,3.03,0.72,...,-67,71,86,4.52,4.51,329.7,61.8,635,Ancient,6665
1,648,240.4,582,4.08,338.5,27.1,35,871,2.96,0.48,...,28,77,69,4.63,4.67,250.8,71.7,635,Ancient,7532
2,696,208.5,693,4.98,353.1,25.2,41,1103,3.18,0.64,...,3,117,103,5.72,5.21,347.8,77.9,583,Dust2,6665
3,2180,256.0,2019,5.39,373.5,25.7,119,3181,3.41,0.62,...,125,312,296,5.48,5.18,387.8,80.7,583,Dust2,7532
4,801,247.2,626,6.41,383.8,22.2,45,1125,3.57,0.62,...,175,134,91,8.71,5.64,391.2,78.0,439,Dust2,4608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,381,217.3,374,5.19,347.0,26.4,21,583,3.27,0.54,...,7,55,56,5.20,4.90,409.5,73.4,284,Ancient,10503
1482,573,243.8,553,5.21,368.7,22.2,30,860,3.32,0.60,...,20,83,89,4.95,4.90,375.8,70.5,27,Vertigo,7020
1483,899,210.8,691,6.55,411.5,35.3,44,1157,3.83,0.83,...,131,103,100,5.38,5.08,353.1,63.5,27,Vertigo,8297
1484,1027,234.1,806,6.42,401.4,27.8,54,1344,3.80,0.64,...,177,106,71,7.73,5.68,400.1,76.5,237,Ancient,7020


In [37]:
df = train.merge(stats_sum.add_suffix('_team1'),left_on=['map_id','team1_id'],right_on=['map_id_team1','team_id_team1']) \
          .merge(stats_sum.add_suffix('_team2'),left_on=['map_id','team2_id'],right_on=['map_id_team2','team_id_team2'])


In [38]:
df

,map_id,team1_id,team2_id,map_name,who_win,total_kills_team1,headshots_team1,total_deaths_team1,kd_ratio_team1,damage_per_round_team1,...,kill_death_difference_team2,total_opening_kills_team2,total_opening_deaths_team2,opening_kill_ratio_team2,opening_kill_rating_team2,team_win_percent_after_first_kill_team2,first_kill_in_won_rounds_team2,map_id_team2,map_name_team2,team_id_team2
0,289,6665,7718,Ancient,0,1348,199.0,1280,5.38,379.1,...,-32,92,91,5.88,5.01,322.4,71.4,289,Ancient,7718
1,715,4411,10577,Inferno,0,1234,225.6,1321,4.68,357.9,...,131,299,253,6.94,5.44,365.0,77.2,715,Inferno,10577
2,157,11251,9455,Nuke,1,2029,236.1,1792,5.67,383.4,...,-11,130,112,6.48,5.25,322.9,75.4,157,Nuke,9455
3,524,4608,7532,Mirage,0,1754,238.3,1532,5.71,375.7,...,44,203,207,4.80,4.87,354.0,67.7,524,Mirage,7532
4,404,8637,6667,Overpass,1,1268,224.4,1210,5.27,371.5,...,40,108,118,5.90,4.96,391.8,74.8,404,Overpass,6667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,709,6667,4773,Inferno,0,1772,239.2,1646,5.46,367.4,...,121,176,122,7.97,5.65,347.6,79.3,709,Inferno,4773
709,528,9215,5995,Ancient,1,1556,235.3,1303,5.95,387.1,...,41,60,59,4.04,4.27,305.4,65.5,528,Ancient,5995
710,163,4869,9565,Mirage,1,2733,224.2,2615,5.23,378.2,...,-79,200,227,4.71,4.79,355.5,71.9,163,Mirage,9565
711,96,10426,4991,Nuke,1,2783,221.6,2422,5.76,397.5,...,60,202,233,5.22,4.90,386.9,71.7,96,Nuke,4991


In [39]:
cols_to_drop = ['who_win','map_id',
                'map_id_team1','map_id_team2',
                'team_id_team1','team_id_team2',
                'map_name_team1','map_name_team2',
               # 'p1_id_team1','p1_id_team2',
               # 'p2_id_team1','p2_id_team2',
               # 'p3_id_team1','p3_id_team2',
               # 'p4_id_team1','p4_id_team2',
               # 'p5_id_team1','p5_id_team2'
               ]

In [40]:
X = df.drop(columns=cols_to_drop)
y = df['who_win']

In [41]:
cat_features = ['team1_id','team2_id','map_name']

In [42]:
# cv_func(X,y,cat_features)

## 3 train.csv only ~.72 - best

In [44]:
train.head(2)

,map_id,team1_id,team2_id,map_name,who_win
0,289,6665,7718,Ancient,0
1,715,4411,10577,Inferno,0


In [85]:
X = train.drop(columns=['map_id','who_win'])
y = train['who_win']

cat_features = ['team1_id','team2_id','map_name']

In [86]:
cv_func(X,y,cat_features)

[I 2022-12-04 21:31:30,506] A new study created in memory with name: no-name-f8c2f36a-217f-4734-986e-0aeae8f9ba1e
C:\Users\SnakeRZR\Desktop\projects\envs\cs_tournament\lib\site-packages\optuna\study\study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2022-12-04 21:31:45,734] Trial 2 finished with value: 0.6512734362037497 and parameters: {'depth': 2, 'learning_rate': 0.014147373460280891, 'n_estimators': 243}. Best is trial 2 with value: 0.6512734362037497.


[0.6886510701841713, 0.6729591836734694, 0.6808943089430894, 0.654979674796748, 0.6961382113821137, 0.6692073170731707, 0.5757113821138211, 0.5716463414634146]


[I 2022-12-04 21:32:06,528] Trial 0 finished with value: 0.6856276962004314 and parameters: {'depth': 6, 'learning_rate': 0.12597824499415414, 'n_estimators': 245}. Best is trial 0 with value: 0.6856276962004314.


[0.721005475360876, 0.6979591836734693, 0.7942073170731708, 0.6742886178861789, 0.7017276422764227, 0.6803861788617886, 0.6285569105691057, 0.586890243902439]


[I 2022-12-04 21:32:21,503] Trial 4 finished with value: 0.6862838580554173 and parameters: {'depth': 5, 'learning_rate': 0.023692029502905464, 'n_estimators': 299}. Best is trial 4 with value: 0.6862838580554173.


...

[0.7200099552015928, 0.7030612244897959, 0.763719512195122, 0.7535569105691058, 0.7428861788617886, 0.720020325203252, 0.6737804878048781, 0.6001016260162603]


[I 2022-12-04 21:42:22,961] Trial 62 finished with value: 0.6735780135224821 and parameters: {'depth': 6, 'learning_rate': 0.1930782876005761, 'n_estimators': 488}. Best is trial 60 with value: 0.7194287166085946.


[0.7324539571926332, 0.6678571428571429, 0.7154471544715447, 0.7022357723577235, 0.7367886178861789, 0.6620934959349594, 0.5492886178861788, 0.6224593495934959]
Best params: {'depth': 3, 'learning_rate': 0.17580816107502642, 'n_estimators': 504}
Best roc_auc: 0.7194287166085946


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/8]

bestTest = 0.5925905229
bestIteration = 86

Training on fold [1/8]

bestTest = 0.6414491909
bestIteration = 50

Training on fold [2/8]

bestTest = 0.6274675805
bestIteration = 87

Training on fold [3/8]

bestTest = 0.5670041243
bestIteration = 275

Training on fold [4/8]

bestTest = 0.6128417697
bestIteration = 58

Training on fold [5/8]

bestTest = 0.6264513439
bestIteration = 69

Training on fold [6/8]

bestTest = 0.6030425702
bestIteration = 412

Training on fold [7/8]

bestTest = 0.5550934133
bestIteration = 173



## 4 train.csv only, logreg + ohe for teams and map names

In [ ]:
train.head()

In [ ]:
X = train.drop(columns=['map_id','who_win'])
y = train['who_win']

In [ ]:
ohe = OneHotEncoder()
X = ohe.fit_transform(X)
X = pd.DataFrame(X.toarray(),columns=ohe.get_feature_names_out())

In [ ]:
logreg = LogisticRegression()
logreg_cv = cross_val_score(logreg,X,y,cv=8,scoring='roc_auc')

### result

In [ ]:
logreg_cv

## 5 mean, std for each stat for a team

In [47]:
stats_statistics = players_feats[col_names[-3:]].copy()

for col_name in col_names[1:-3]:
    median = players_feats.filter(regex=col_name,axis=1).mean(axis=1)
    median.name = col_name + '_mean'
    
    std = players_feats.filter(regex=col_name,axis=1).std(axis=1)
    std.name = col_name + '_std'
    
    # display(median)
    # display(std)
    
    # qs = players_feats.filter(regex=col_name,axis=1).quantile([.01,.99],axis=1)
    # qs.name = col_name + '_q'
    
    stats_statistics = pd.concat([stats_statistics,median,std],axis=1)

stats_statistics.head()

,map_id,map_name,team_id,total_kills_mean,total_kills_std,headshots_mean,headshots_std,total_deaths_mean,total_deaths_std,kd_ratio_mean,...,total_opening_deaths_mean,total_opening_deaths_std,opening_kill_ratio_mean,opening_kill_ratio_std,opening_kill_rating_mean,opening_kill_rating_std,team_win_percent_after_first_kill_mean,team_win_percent_after_first_kill_std,first_kill_in_won_rounds_mean,first_kill_in_won_rounds_std
0,635,Ancient,6665,98.4,34.420924,44.96,11.573807,111.4,33.842281,0.876,...,17.2,6.300794,0.904,0.684967,0.902,0.237844,65.94,16.172755,12.36,8.027951
1,635,Ancient,7532,129.6,209.397469,48.08,10.581446,116.4,141.787870,0.816,...,13.8,11.755850,0.926,0.476267,0.934,0.143631,50.16,32.308404,14.34,9.691646
2,583,Dust2,6665,139.2,41.649730,41.70,8.779806,138.6,36.280849,0.996,...,20.6,2.701851,1.144,0.282454,1.042,0.085849,69.56,15.634673,15.58,4.326315
3,583,Dust2,7532,436.0,81.749618,51.20,15.260079,403.8,35.059949,1.078,...,59.2,9.934787,1.096,0.422410,1.036,0.176153,77.56,7.550695,16.14,5.936582
4,439,Dust2,4608,160.2,12.296341,49.44,12.690075,125.2,8.105554,1.282,...,18.2,7.791020,1.742,0.904279,1.128,0.190971,78.24,7.185263,15.60,6.118823


In [ ]:
df = train.merge(stats_statistics.add_suffix('_team1'),left_on=['map_id','team1_id'],right_on=['map_id_team1','team_id_team1']).merge(stats_statistics.add_suffix('_team2'),left_on=['map_id','team2_id'],right_on=['map_id_team2','team_id_team2'])


In [ ]:
df

In [ ]:
cols_to_drop = ['who_win','map_id',
                'map_id_team1','map_id_team2',
                'team_id_team1','team_id_team2',
                'map_name_team1','map_name_team2',
               # 'p1_id_team1','p1_id_team2',
               # 'p2_id_team1','p2_id_team2',
               # 'p3_id_team1','p3_id_team2',
               # 'p4_id_team1','p4_id_team2',
               # 'p5_id_team1','p5_id_team2'
               ]

In [ ]:
X = df.drop(columns=cols_to_drop)
y = df['who_win']

In [ ]:
# cv_func(X,y,cat_features,4)

## 6 mean,std stat difference

In [ ]:
team1 = train.merge(stats_statistics,left_on=['map_id','team1_id'],right_on=['map_id','team_id']).iloc[:,8:]
team2 = train.merge(stats_statistics,left_on=['map_id','team2_id'],right_on=['map_id','team_id']).iloc[:,8:]

In [ ]:
stats_coef = team1/team2

In [ ]:
df = pd.concat([train,stats_coef],axis=1)
df

In [ ]:
df.columns

In [ ]:
cols_to_drop = ['map_id','who_win']

In [ ]:
X = df.drop(columns=cols_to_drop)
y = df['who_win']

In [ ]:
X.columns

In [ ]:
cat_features = ['team1_id', 'team2_id','map_name']

In [ ]:
cv_func(X,y,cat_features)

## 7 train + kd

In [ ]:
train

In [50]:
team1 = train.merge(players_feats,left_on=['map_id','team1_id'],right_on=['map_id','team_id'])#.iloc[:,8:]
team2 = train.merge(players_feats,left_on=['map_id','team2_id'],right_on=['map_id','team_id'])#.iloc[:,8:]

In [ ]:
team1_kd = np.sort(team1.filter(regex='kd_r').values)
team2_kd = np.sort(team2.filter(regex='kd_r').values)
kd = team1_kd - team2_kd
kd

In [ ]:
kd_sum = np.sum(kd,axis=1)

In [ ]:
df = train.copy()
df['kd_sum'] = kd_sum
df

In [ ]:
X = df.drop(columns=['map_id','who_win'])
y = df['who_win']


In [ ]:
X

In [ ]:
cat_features = ['team1_id','team2_id','map_name']

In [ ]:
cv_func(X,y,cat_features)

## 8 Features selection (wrapper) (train + every players' stat)

In [49]:
col_names[1:-3]

['total_kills',
 'headshots',
 'total_deaths',
 'kd_ratio',
 'damage_per_round',
 'grenade_damage_per_round',
 'maps_played',
 'rounds_played',
 'kills_per_round',
 'assists_per_round',
 'deaths_per_round',
 'saved_by_teammate_per_round',
 'saved_teammates_per_round',
 'rating',
 'kill_death',
 'kill_round',
 'rounds_with_kills',
 'kill_death_difference',
 'total_opening_kills',
 'total_opening_deaths',
 'opening_kill_ratio',
 'opening_kill_rating',
 'team_win_percent_after_first_kill',
 'first_kill_in_won_rounds']

In [ ]:
team1 = train.merge(players_feats,left_on=['map_id','team1_id'],right_on=['map_id','team_id'])#.iloc[:,8:]
team2 = train.merge(players_feats,left_on=['map_id','team2_id'],right_on=['map_id','team_id'])#.iloc[:,8:]



In [68]:
results = {}

for col_name in col_names[1:-3]:
    p_col_names = [x + col_name for x in ['p1_','p2_','p3_','p4_','p5_']]
    
    team1_stat = np.sort(team1[p_col_names].values)
    team2_stat = np.sort(team2[p_col_names].values)
    
    # display(team1[p_col_names])
    # display(team1_stat)
    # display(team2[p_col_names])
    # display(team2_stat)
    
    stat = team1_stat - team2_stat
    stat_sum = np.sum(stat,axis=1)
    # display(stat_sum)
    
    df = train.copy()
    df[col_name] = stat_sum
    
    # display(df)
    
    X = df.drop(columns=['map_id','who_win'])
    y = df['who_win']
    # display(X)
    cat_features = ['team1_id','team2_id','map_name']
    
    
    print(col_name)
    result = cv_func(X,y,cat_features,plot_best=True,return_best=True)
    results[col_name] = result
    print('='*20)
    print('='*20)    
    print('='*20)    

[I 2022-12-04 16:34:26,633] A new study created in memory with name: no-name-5e6111f2-0d8f-48c2-9672-753ab5fa8be4


total_kills


C:\Users\SnakeRZR\Desktop\projects\envs\cs_tournament\lib\site-packages\optuna\study\study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2022-12-04 16:34:41,160] Trial 1 finished with value: 0.6394630413140867 and parameters: {'depth': 4, 'learning_rate': 0.020386222315880064, 'n_estimators': 133}. Best is trial 1 with value: 0.6394630413140867.


[0.6958685913389746, 0.6545918367346939, 0.6341463414634146, 0.6595528455284554, 0.6935975609756098, 0.6453252032520326, 0.5614837398373984, 0.5711382113821138]


[I 2022-12-04 16:34:54,480] Trial 3 finished with value: 0.6904375622200098 and parameters: {'depth': 3, 'learning_rate': 0.09282687880817829, 'n_estimators': 261}. Best is trial 3 with value: 0.6904375622200098.


...

[0.7705326032852166, 0.7326530612244898, 0.7352642276422765, 0.7296747967479675, 0.7184959349593496, 0.6803861788617886, 0.6615853658536586, 0.5947662601626017]


[I 2022-12-04 21:05:29,151] Trial 39 finished with value: 0.7033819167911067 and parameters: {'depth': 3, 'learning_rate': 0.08567234133541939, 'n_estimators': 817}. Best is trial 37 with value: 0.7087359486477519.


[0.726231956197113, 0.7275510204081633, 0.7327235772357723, 0.7576219512195121, 0.7240853658536586, 0.6915650406504065, 0.649390243902439, 0.6178861788617886]
Best params: {'depth': 5, 'learning_rate': 0.08042283080903777, 'n_estimators': 534}
Best roc_auc: 0.7087359486477519


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/8]

bestTest = 0.5937167084
bestIteration = 88

Training on fold [1/8]

bestTest = 0.6681234716
bestIteration = 47

Training on fold [2/8]

bestTest = 0.6498939125
bestIteration = 42

Training on fold [3/8]

bestTest = 0.5691985722
bestIteration = 313

Training on fold [4/8]

bestTest = 0.621591437
bestIteration = 225

Training on fold [5/8]

bestTest = 0.6398464896
bestIteration = 92

Training on fold [6/8]

bestTest = 0.6531784816
bestIteration = 101

Training on fold [7/8]

bestTest = 0.5952421655
bestIteration = 157



In [76]:
results = pd.DataFrame(results).T

In [78]:
more_than_65 = [0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0]

results['more_than_65'] = more_than_65

In [79]:
results

,0,1,more_than_65
total_kills,0.712744,"{'depth': 3, 'learning_rate': 0.11635793031064...",0
headshots,0.700309,"{'depth': 4, 'learning_rate': 0.02542323229535...",1
total_deaths,0.699469,"{'depth': 4, 'learning_rate': 0.07644464790115...",1
kd_ratio,0.692934,"{'depth': 3, 'learning_rate': 0.15755462020610...",1
damage_per_round,0.701621,"{'depth': 3, 'learning_rate': 0.07076103430926...",0
grenade_damage_per_round,0.698597,"{'depth': 4, 'learning_rate': 0.15795944366111...",0
maps_played,0.694842,"{'depth': 4, 'learning_rate': 0.01977468560412...",0
rounds_played,0.707731,"{'depth': 3, 'learning_rate': 0.14723637596817...",0
kills_per_round,0.704371,"{'depth': 3, 'learning_rate': 0.11225249959257...",0
assists_per_round,0.702289,"{'depth': 3, 'learning_rate': 0.19767541647847...",0


In [80]:
# results.to_csv('features_selection/result.csv')

# Final models

In [81]:
test = pd.read_csv('data/test.csv')

team1_test = test.merge(players_feats,left_on=['map_id','team1_id'],right_on=['map_id','team_id'])
team2_test = test.merge(players_feats,left_on=['map_id','team2_id'],right_on=['map_id','team_id'])

## train.csv only

In [89]:
X_train = train.drop(columns=['map_id','who_win'])
y_train = train['who_win']

cat_features = ['team1_id','team2_id','map_name']

X_test = test.drop(columns=['index','map_id'])

In [93]:
params = {'depth': 3, 'learning_rate': 0.17580816107502642, 'n_estimators': 504}
params['loss_function'] = 'Logloss'
params['custom_loss'] = 'AUC'
params["verbose"] = False

model = cb.CatBoostClassifier(**params)
model.fit(X_train,y_train,cat_features=cat_features)
prediction_1 = model.predict_proba(X_test)[:,1]

In [96]:
pd.DataFrame(prediction_1).to_csv('prediction_1.csv')

## train.csv + total_deaths sum of differences

In [110]:
# train features preparation
team1_td = np.sort(team1.filter(like='total_death').values)
team2_td = np.sort(team2.filter(like='total_death').values)

td = team1_td - team2_td
stat_sum = np.sum(td,axis=1)

df = train.copy()
df['total_deaths'] = stat_sum

X_train = df.drop(columns=['map_id','who_win'])
y_train = df['who_win']


In [111]:
# test features preparation
team1_td = np.sort(team1_test.filter(like='total_death').values)
team2_td = np.sort(team2_test.filter(like='total_death').values)

td = team1_td - team2_td
stat_sum = np.sum(td,axis=1)

df_test = test.copy()
df_test['total_deaths'] = stat_sum

X_test = df_test.drop(columns=['index','map_id'])

In [112]:
cat_features = ['team1_id','team2_id','map_name']

In [113]:
params = {'depth': 4, 'learning_rate': 0.07644464790115962, 'n_estimators': 480}
params['loss_function'] = 'Logloss'
params['custom_loss'] = 'AUC'
params["verbose"] = False

model = cb.CatBoostClassifier(**params)
model.fit(X_train,y_train,cat_features=cat_features)
prediction_2 = model.predict_proba(X_test)[:,1]

In [115]:
pd.DataFrame(prediction_2).to_csv('prediction_2.csv')

## train.csv + sums of differences: 

In [128]:
results[results['more_than_65'] == 1]

,0,1,more_than_65
headshots,0.700309,"{'depth': 4, 'learning_rate': 0.02542323229535...",1
total_deaths,0.699469,"{'depth': 4, 'learning_rate': 0.07644464790115...",1
kd_ratio,0.692934,"{'depth': 3, 'learning_rate': 0.15755462020610...",1
total_opening_kills,0.710302,"{'depth': 4, 'learning_rate': 0.08369492628770...",1
total_opening_deaths,0.705769,"{'depth': 3, 'learning_rate': 0.11827197711048...",1


In [129]:
cols = results[results['more_than_65'] == 1].index.tolist()

In [130]:
df = train.copy()

for col_name in cols:
    p_col_names = [x + col_name for x in ['p1_','p2_','p3_','p4_','p5_']]
    
    team1_stat = np.sort(team1[p_col_names].values)
    team2_stat = np.sort(team2[p_col_names].values)
    
    stat = team1_stat - team2_stat
    stat_sum = np.sum(stat,axis=1)

    df[col_name] = stat_sum
    

    
  

In [131]:
df.head(1)

,map_id,team1_id,team2_id,map_name,who_win,headshots,total_deaths,kd_ratio,total_opening_kills,total_opening_deaths
0,289,6665,7718,Ancient,0,-26.3,665,0.55,116,111


In [132]:
X_train = df.drop(columns=['map_id','who_win'])
y_train = df['who_win']

In [133]:
cat_features = ['team1_id','team2_id','map_name']

In [134]:
df = test.copy()

for col_name in cols:
    p_col_names = [x + col_name for x in ['p1_','p2_','p3_','p4_','p5_']]
    
    team1_stat = np.sort(team1_test[p_col_names].values)
    team2_stat = np.sort(team2_test[p_col_names].values)
    
    stat = team1_stat - team2_stat
    stat_sum = np.sum(stat,axis=1)

    df[col_name] = stat_sum


In [135]:
X_test = df.drop(columns=['index','map_id'])

In [136]:
cv_func(X_train,y_train,cat_features)

[I 2022-12-04 22:04:20,834] A new study created in memory with name: no-name-6e412a3e-7ee2-400a-ab2b-120bd427c959
C:\Users\SnakeRZR\Desktop\projects\envs\cs_tournament\lib\site-packages\optuna\study\study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2022-12-04 22:04:30,354] Trial 2 finished with value: 0.6384734839057573 and parameters: {'depth': 4, 'learning_rate': 0.01060441149164382, 'n_estimators': 107}. Best is trial 2 with value: 0.6384734839057573.


[0.6276754604280737, 0.6076530612244897, 0.6321138211382114, 0.7179878048780488, 0.7103658536585366, 0.6316056910569106, 0.5579268292682927, 0.6224593495934959]


[I 2022-12-04 22:04:36,534] Trial 0 finished with value: 0.6379067633150821 and parameters: {'depth': 5, 'learning_rate': 0.12556606078208826, 'n_estimators': 136}. Best is trial 2 with value: 0.6384734839057573.


[0.6326530612244898, 0.6428571428571428, 0.6686991869918699, 0.6885162601626016, 0.6875, 0.5431910569105691, 0.6153455284552846, 0.6244918699186992]


[I 2022-12-04 22:04:41,193] Trial 4 finished with value: 0.660857650987224 and parameters: {'depth': 3, 'learning_rate': 0.04905048338701537, 'n_estimators': 159}. Best is trial 4 with value: 0.660857650987224.


...

[0.7063215530114485, 0.6448979591836734, 0.7184959349593495, 0.7352642276422765, 0.7169715447154472, 0.657520325203252, 0.6270325203252033, 0.6366869918699187]


[I 2022-12-04 22:15:17,532] Trial 52 finished with value: 0.6814351045296168 and parameters: {'depth': 5, 'learning_rate': 0.028129771591119727, 'n_estimators': 752}. Best is trial 39 with value: 0.6840970839555334.


[0.704330512692882, 0.6739795918367347, 0.7113821138211383, 0.7459349593495935, 0.7093495934959351, 0.641260162601626, 0.6438008130081301, 0.6214430894308943]
Best params: {'depth': 5, 'learning_rate': 0.018518126630923688, 'n_estimators': 390}
Best roc_auc: 0.6840970839555334


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/8]

bestTest = 0.6194416767
bestIteration = 358

Training on fold [1/8]

bestTest = 0.6619508173
bestIteration = 388

Training on fold [2/8]

bestTest = 0.6464695173
bestIteration = 283

Training on fold [3/8]

bestTest = 0.5815107338
bestIteration = 383

Training on fold [4/8]

bestTest = 0.6215465912
bestIteration = 383

Training on fold [5/8]

bestTest = 0.6303342352
bestIteration = 346

Training on fold [6/8]

bestTest = 0.6463414466
bestIteration = 346

Training on fold [7/8]

bestTest = 0.6199888992
bestIteration = 386



In [137]:
params = {'depth': 5, 'learning_rate': 0.018518126630923688, 'n_estimators': 390}
params['loss_function'] = 'Logloss'
params['custom_loss'] = 'AUC'
params["verbose"] = False

model = cb.CatBoostClassifier(**params)
model.fit(X_train,y_train,cat_features=cat_features)
prediction_3 = model.predict_proba(X_test)[:,1]

In [139]:
pd.DataFrame(prediction_3).to_csv('prediction_3.csv')